In [ ]:
!nvcc --version
%pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [2]:
%%cuda
#include <stdio.h>
#include <stdlib.h>

#define N 8
#define BLOCK_SIZE 4

__global__ 
void matrixMulShared(float *A, float *B, float *C, int width) {
    
    __shared__ float tileA[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ float tileB[BLOCK_SIZE][BLOCK_SIZE];

    
    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;

    float value = 0.0f;

    
    for (int t = 0; t < width / BLOCK_SIZE; ++t) {
        
        if (row < width && t * BLOCK_SIZE + threadIdx.x < width)
            tileA[threadIdx.y][threadIdx.x] = A[row * width + t * BLOCK_SIZE + threadIdx.x];
        else
            tileA[threadIdx.y][threadIdx.x] = 0;

        if (t * BLOCK_SIZE + threadIdx.y < width && col < width)
            tileB[threadIdx.y][threadIdx.x] = B[(t * BLOCK_SIZE + threadIdx.y) * width + col];
        else
            tileB[threadIdx.y][threadIdx.x] = 0;

        __syncthreads();

        for (int i = 0; i < BLOCK_SIZE; ++i)
            value += tileA[threadIdx.y][i] * tileB[i][threadIdx.x];

        __syncthreads();
    }

    if (row < width && col < width)
        C[row * width + col] = value;
}

int main() {
    int size = N * N;
    size_t sz = size * sizeof(float);

    float *h_A = (float *)malloc(sz);
    float *h_B = (float *)malloc(sz);
    float *h_C = (float *)malloc(sz);

    for (int i = 0; i < size; ++i) {
        h_A[i] = i % 10;
        h_B[i] = (i % 5) + 1;
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, sz);
    cudaMalloc(&d_B, sz);
    cudaMalloc(&d_C, sz);

    cudaMemcpy(d_A, h_A, sz, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sz, cudaMemcpyHostToDevice);

    dim3 threads(BLOCK_SIZE, BLOCK_SIZE);
    dim3 blocks((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    matrixMulShared<<<blocks, threads>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(h_C, d_C, sz, cudaMemcpyDeviceToHost);

    printf("A:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++)
            printf("%5.1f ", h_A[i * N + j]);
        printf("\n");
    }

    printf("\nB:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++)
            printf("%5.1f ", h_B[i * N + j]);
        printf("\n");
    }

    printf("\nC = A x B:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++)
            printf("%5.1f ", h_C[i * N + j]);
        printf("\n");
    }

    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}


A:
  0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0 
  8.0   9.0   0.0   1.0   2.0   3.0   4.0   5.0 
  6.0   7.0   8.0   9.0   0.0   1.0   2.0   3.0 
  4.0   5.0   6.0   7.0   8.0   9.0   0.0   1.0 
  2.0   3.0   4.0   5.0   6.0   7.0   8.0   9.0 
  0.0   1.0   2.0   3.0   4.0   5.0   6.0   7.0 
  8.0   9.0   0.0   1.0   2.0   3.0   4.0   5.0 
  6.0   7.0   8.0   9.0   0.0   1.0   2.0   3.0 

B:
  1.0   2.0   3.0   4.0   5.0   1.0   2.0   3.0 
  4.0   5.0   1.0   2.0   3.0   4.0   5.0   1.0 
  2.0   3.0   4.0   5.0   1.0   2.0   3.0   4.0 
  5.0   1.0   2.0   3.0   4.0   5.0   1.0   2.0 
  3.0   4.0   5.0   1.0   2.0   3.0   4.0   5.0 
  1.0   2.0   3.0   4.0   5.0   1.0   2.0   3.0 
  4.0   5.0   1.0   2.0   3.0   4.0   5.0   1.0 
  2.0   3.0   4.0   5.0   1.0   2.0   3.0   4.0 

C = A x B:
 78.0  91.0  84.0  92.0  75.0  78.0  91.0  84.0 
 84.0 111.0  78.0 100.0 107.0  84.0 111.0  78.0 
110.0 101.0  92.0 128.0 109.0 110.0 101.0  92.0 
106.0 111.0 126.0 126.0 131.0 106.0 111.0 126.0 
1